<a href="https://colab.research.google.com/github/baochi0212/Optimization/blob/master/clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/louiezzang/vehicle-routing-optimization/raw/master/data/locations.csv

--2021-12-18 07:54:07--  https://github.com/louiezzang/vehicle-routing-optimization/raw/master/data/locations.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/louiezzang/vehicle-routing-optimization/master/data/locations.csv [following]
--2021-12-18 07:54:07--  https://raw.githubusercontent.com/louiezzang/vehicle-routing-optimization/master/data/locations.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13496 (13K) [text/plain]
Saving to: ‘locations.csv’

locations.csv       100%[===================>]  13.18K  --.-KB/s    in 0s      

2021-12-18 07:54:07 (37.5 MB/s) - ‘locations.csv’ saved

In [ ]:
#copy tsp.py to this directory 

In [22]:
import pandas as pd
import sklearn
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
import numpy as np
import random
from tsp import TSP

In [3]:
data = pd.read_csv('locations.csv')

In [4]:
data

,11.552931,104.933636,0
0,11.452374,104.807993,1
1,11.469027,104.707982,1
2,11.490957,104.833844,1
3,11.511254,104.773356,1
4,11.511943,104.877798,1
...,...,...,...
531,11.621318,104.923603,1
532,11.639438,105.008719,1
533,11.662220,104.801737,1
534,11.662220,104.801737,1


In [5]:
data.columns

Index(['11.552931', '104.933636', '0'], dtype='object')

In [6]:
data['x'] = data['11.552931']
data['y'] = data['104.933636']

In [7]:
data = data.drop(['11.552931', '104.933636', '0'], axis=1)

In [8]:
fig = px.scatter(x=data['x'], y=data['y'])
fig.show()

In [9]:
model = KMeans(n_clusters=3, random_state=3)

In [10]:
model.fit(data)

KMeans(n_clusters=3, random_state=3)

In [11]:
data['labels'] = model.labels_

In [12]:
data

,x,y,labels
0,11.452374,104.807993,2
1,11.469027,104.707982,2
2,11.490957,104.833844,2
3,11.511254,104.773356,2
4,11.511943,104.877798,2
...,...,...,...
531,11.621318,104.923603,1
532,11.639438,105.008719,0
533,11.662220,104.801737,1
534,11.662220,104.801737,1


In [13]:
fig = px.scatter(data, x='x', y='y', color='labels')
fig.show()

In [14]:
model = GaussianMixture(n_components=3, random_state=3)

In [15]:
gm = model.fit(data)

In [16]:
data['labels2'] = gm.predict(data)

In [17]:
fig = px.scatter(data, x='x', y='y', color='labels2')
fig.show()

In [18]:
def distance_matrix(list):
  matrix = np.random.rand(len(list), len(list))
  for i in range(len(list)):
    for j in range(len(list)):
      x1, y1 = list[i]
      x2, y2 = list[j]
      matrix[i, j] = ((x1 - x2)**2 + (y1 - y2)**2)**0.5
  return matrix
 

dict = {}
for i in range(len(data)):
  label = data.iloc[i].labels
  if label in dict:
    dict[label].append((data.iloc[i].x, data.iloc[i].y))
  else:
    dict[label] = [(data.iloc[i].x, data.iloc[i].y)]

for list in dict:
  dict[list] = distance_matrix(dict[list])
  print(dict[list].shape)

(33, 33)
(397, 397)
(106, 106)


In [19]:
list = []
for i in range(len(data)):
  list.append(random.uniform(0, 1))

data['time'] = list
data

,x,y,labels,labels2,time
0,11.452374,104.807993,2,2,0.512857
1,11.469027,104.707982,2,2,0.686251
2,11.490957,104.833844,2,2,0.098123
3,11.511254,104.773356,2,2,0.649172
4,11.511943,104.877798,2,2,0.569242
...,...,...,...,...,...
531,11.621318,104.923603,1,1,0.101523
532,11.639438,105.008719,0,0,0.354273
533,11.662220,104.801737,1,1,0.448636
534,11.662220,104.801737,1,1,0.834938


In [28]:
#get the loads
loads = {}
for i in range(len(data)):
  if data.loc[i].labels in loads:
    loads[data.loc[i].labels].append(data.loc[i].time)
  else:
    loads[data.loc[i].labels] = [data.loc[i].time]
print(len(loads[0]))
print(len(dict[0].tolist()))

397
397


In [36]:
result = []

for list in dict:
  #distance matrix:
  distance_mat = dict[list]
  #load
  load = loads[list]
  tsp = TSP(distance_mat, load, len(load) - 1)
  result.append(tsp.main(1000, 1, 0.5))
  print(distance_mat.shape)
  print(len(load))


(33, 33)
33
(397, 397)
397
(106, 106)
106


In [38]:
result[0]

([0,
  1,
  2,
  8,
  4,
  11,
  6,
  7,
  18,
  9,
  10,
  31,
  20,
  13,
  14,
  25,
  28,
  17,
  30,
  19,
  12,
  21,
  22,
  23,
  24,
  15,
  27,
  26,
  16,
  29,
  3,
  32,
  5,
  0],
 18.959458989780533)